In [1]:
# -*- coding: utf-8 -*-
""" Simple implementation of Generative Adversarial Neural Network """

import numpy as np

from IPython.core.debugger import Tracer

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential
from keras.optimizers import Adam

import matplotlib.pyplot as plt
plt.switch_backend('agg')   # allows code to run without a system DISPLAY

Using TensorFlow backend.


In [11]:
import os

In [12]:
class GAN(object):
    """ Generative Adversarial Network class """
    def __init__(self, width=28, height=28, channels=1):

        self.width = width
        self.height = height
        self.channels = channels

        self.shape = (self.width, self.height, self.channels)

        self.optimizer = Adam(lr=0.0002, beta_1=0.5, decay=8e-8)

        self.G = self.__generator()
        self.G.compile(loss='binary_crossentropy', optimizer=self.optimizer)

        self.D = self.__discriminator()
        self.D.compile(loss='binary_crossentropy', optimizer=self.optimizer, metrics=['accuracy'])

        self.stacked_generator_discriminator = self.__stacked_generator_discriminator()

        self.stacked_generator_discriminator.compile(loss='binary_crossentropy', optimizer=self.optimizer)


    def __generator(self):
        """ Declare generator """

        model = Sequential()
        model.add(Dense(256, input_shape=(100,)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(self.width  * self.height * self.channels, activation='tanh'))
        model.add(Reshape((self.width, self.height, self.channels)))

        return model

    def __discriminator(self):
        """ Declare discriminator """

        model = Sequential()
        model.add(Flatten(input_shape=self.shape))
        model.add(Dense((self.width * self.height * self.channels), input_shape=self.shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(np.int64((self.width * self.height * self.channels)/2)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        return model

    def __stacked_generator_discriminator(self):

        self.D.trainable = False

        model = Sequential()
        model.add(self.G)
        model.add(self.D)

        return model

    def train(self, X_train, epochs=20000, batch = 32, save_interval = 100):

        for cnt in range(epochs):

            ## train discriminator
            random_index = np.random.randint(0, len(X_train) - np.int64(batch/2))
            legit_images = X_train[random_index : random_index + np.int64(batch/2)].reshape(np.int64(batch/2), self.width, self.height, self.channels)

            gen_noise = np.random.normal(0, 1, (np.int64(batch/2), 100))
            syntetic_images = self.G.predict(gen_noise)

            x_combined_batch = np.concatenate((legit_images, syntetic_images))
            y_combined_batch = np.concatenate((np.ones((np.int64(batch/2), 1)), np.zeros((np.int64(batch/2), 1))))

            d_loss = self.D.train_on_batch(x_combined_batch, y_combined_batch)


            # train generator

            noise = np.random.normal(0, 1, (batch, 100))
            y_mislabled = np.ones((batch, 1))

            g_loss = self.stacked_generator_discriminator.train_on_batch(noise, y_mislabled)

            print ('epoch: %d, [Discriminator :: d_loss: %f], [ Generator :: loss: %f]' % (cnt, d_loss[0], g_loss))

            if cnt % save_interval == 0:
                self.plot_images(save2file=True, step=cnt)


    def plot_images(self, save2file=False, samples=16, step=0):
        ''' Plot and generated images '''
        if not os.path.exists("./images"):
            os.makedirs("./images")
        filename = "./images/mnist_%d.png" % step
        noise = np.random.normal(0, 1, (samples, 100))

        images = self.G.predict(noise)

        plt.figure(figsize=(10, 10))

        for i in range(images.shape[0]):
            plt.subplot(4, 4, i+1)
            image = images[i, :, :, :]
            image = np.reshape(image, [self.height, self.width])
            plt.imshow(image, cmap='gray')
            plt.axis('off')
        plt.tight_layout()

        if save2file:
            plt.savefig(filename)
            plt.close('all')
        else:
            plt.show()


if __name__ == '__main__':
    (X_train, _), (_, _) = mnist.load_data()

    # Rescale -1 to 1
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)


    gan = GAN()
    gan.train(X_train)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 784)               615440    
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 784)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 392)               307720    
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 392)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 393       
Total params: 923,553
Trainable params: 923,553
Non-trainable params: 0
_________________________________________________________________


/Applications/anaconda3/lib/python3.7/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch: 0, [Discriminator :: d_loss: 0.760951], [ Generator :: loss: 0.863593]


/Applications/anaconda3/lib/python3.7/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch: 1, [Discriminator :: d_loss: 0.364042], [ Generator :: loss: 0.926610]
epoch: 2, [Discriminator :: d_loss: 0.320334], [ Generator :: loss: 1.087552]
epoch: 3, [Discriminator :: d_loss: 0.258466], [ Generator :: loss: 1.269060]
epoch: 4, [Discriminator :: d_loss: 0.180848], [ Generator :: loss: 1.492519]
epoch: 5, [Discriminator :: d_loss: 0.163211], [ Generator :: loss: 1.635348]
epoch: 6, [Discriminator :: d_loss: 0.111189], [ Generator :: loss: 1.846282]
epoch: 7, [Discriminator :: d_loss: 0.117329], [ Generator :: loss: 2.060217]
epoch: 8, [Discriminator :: d_loss: 0.088243], [ Generator :: loss: 2.204510]
epoch: 9, [Discriminator :: d_loss: 0.073497], [ Generator :: loss: 2.352849]
epoch: 10, [Discriminator :: d_loss: 0.071071], [ Generator :: loss: 2.473576]
epoch: 11, [Discriminator :: d_loss: 0.051744], [ Generator :: loss: 2.553621]
epoch: 12, [Discriminator :: d_loss: 0.055755], [ Generator :: loss: 2.655533]
epoch: 13, [Discriminator :: d_loss: 0.055401], [ Generator :

...

epoch: 19501, [Discriminator :: d_loss: 0.640404], [ Generator :: loss: 0.739934]
epoch: 19502, [Discriminator :: d_loss: 0.576042], [ Generator :: loss: 0.675310]
epoch: 19503, [Discriminator :: d_loss: 0.756440], [ Generator :: loss: 0.923603]
epoch: 19504, [Discriminator :: d_loss: 0.709654], [ Generator :: loss: 0.817198]
epoch: 19505, [Discriminator :: d_loss: 0.623650], [ Generator :: loss: 0.884397]
epoch: 19506, [Discriminator :: d_loss: 0.645453], [ Generator :: loss: 0.830458]
epoch: 19507, [Discriminator :: d_loss: 0.647947], [ Generator :: loss: 0.786509]
epoch: 19508, [Discriminator :: d_loss: 0.624925], [ Generator :: loss: 0.789684]
epoch: 19509, [Discriminator :: d_loss: 0.651802], [ Generator :: loss: 0.684657]
epoch: 19510, [Discriminator :: d_loss: 0.838063], [ Generator :: loss: 0.744022]
epoch: 19511, [Discriminator :: d_loss: 0.741003], [ Generator :: loss: 0.839601]
epoch: 19512, [Discriminator :: d_loss: 0.712868], [ Generator :: loss: 0.858337]
epoch: 19513, [D

epoch: 19601, [Discriminator :: d_loss: 0.629535], [ Generator :: loss: 0.815006]
epoch: 19602, [Discriminator :: d_loss: 0.630986], [ Generator :: loss: 0.861323]
epoch: 19603, [Discriminator :: d_loss: 0.554448], [ Generator :: loss: 0.835368]
epoch: 19604, [Discriminator :: d_loss: 0.675607], [ Generator :: loss: 0.773867]
epoch: 19605, [Discriminator :: d_loss: 0.597301], [ Generator :: loss: 0.763098]
epoch: 19606, [Discriminator :: d_loss: 0.598047], [ Generator :: loss: 0.934553]
epoch: 19607, [Discriminator :: d_loss: 0.629055], [ Generator :: loss: 0.701922]
epoch: 19608, [Discriminator :: d_loss: 0.781614], [ Generator :: loss: 0.816736]
epoch: 19609, [Discriminator :: d_loss: 0.671989], [ Generator :: loss: 0.780854]
epoch: 19610, [Discriminator :: d_loss: 0.627082], [ Generator :: loss: 0.748312]
epoch: 19611, [Discriminator :: d_loss: 0.593572], [ Generator :: loss: 0.968787]
epoch: 19612, [Discriminator :: d_loss: 0.713795], [ Generator :: loss: 0.817239]
epoch: 19613, [D

epoch: 19701, [Discriminator :: d_loss: 0.618907], [ Generator :: loss: 0.935241]
epoch: 19702, [Discriminator :: d_loss: 0.720168], [ Generator :: loss: 0.782762]
epoch: 19703, [Discriminator :: d_loss: 0.679410], [ Generator :: loss: 0.744588]
epoch: 19704, [Discriminator :: d_loss: 0.583607], [ Generator :: loss: 0.874762]
epoch: 19705, [Discriminator :: d_loss: 0.657341], [ Generator :: loss: 0.762163]
epoch: 19706, [Discriminator :: d_loss: 0.695383], [ Generator :: loss: 0.709714]
epoch: 19707, [Discriminator :: d_loss: 0.644642], [ Generator :: loss: 0.776565]
epoch: 19708, [Discriminator :: d_loss: 0.585423], [ Generator :: loss: 0.839228]
epoch: 19709, [Discriminator :: d_loss: 0.620509], [ Generator :: loss: 0.828051]
epoch: 19710, [Discriminator :: d_loss: 0.664625], [ Generator :: loss: 0.849331]
epoch: 19711, [Discriminator :: d_loss: 0.638223], [ Generator :: loss: 0.891077]
epoch: 19712, [Discriminator :: d_loss: 0.543749], [ Generator :: loss: 1.090679]
epoch: 19713, [D

epoch: 19801, [Discriminator :: d_loss: 0.623408], [ Generator :: loss: 1.001491]
epoch: 19802, [Discriminator :: d_loss: 0.757518], [ Generator :: loss: 0.781024]
epoch: 19803, [Discriminator :: d_loss: 0.686231], [ Generator :: loss: 0.679772]
epoch: 19804, [Discriminator :: d_loss: 0.639232], [ Generator :: loss: 1.012963]
epoch: 19805, [Discriminator :: d_loss: 0.543081], [ Generator :: loss: 0.978442]
epoch: 19806, [Discriminator :: d_loss: 0.721156], [ Generator :: loss: 0.721347]
epoch: 19807, [Discriminator :: d_loss: 0.661124], [ Generator :: loss: 0.755057]
epoch: 19808, [Discriminator :: d_loss: 0.766765], [ Generator :: loss: 0.784945]
epoch: 19809, [Discriminator :: d_loss: 0.677086], [ Generator :: loss: 0.831710]
epoch: 19810, [Discriminator :: d_loss: 0.665447], [ Generator :: loss: 0.890614]
epoch: 19811, [Discriminator :: d_loss: 0.654326], [ Generator :: loss: 0.895456]
epoch: 19812, [Discriminator :: d_loss: 0.631841], [ Generator :: loss: 0.841430]
epoch: 19813, [D

epoch: 19901, [Discriminator :: d_loss: 0.671902], [ Generator :: loss: 0.879555]
epoch: 19902, [Discriminator :: d_loss: 0.607820], [ Generator :: loss: 0.818181]
epoch: 19903, [Discriminator :: d_loss: 0.672160], [ Generator :: loss: 0.873051]
epoch: 19904, [Discriminator :: d_loss: 0.621446], [ Generator :: loss: 0.895979]
epoch: 19905, [Discriminator :: d_loss: 0.668445], [ Generator :: loss: 0.734489]
epoch: 19906, [Discriminator :: d_loss: 0.677069], [ Generator :: loss: 0.675087]
epoch: 19907, [Discriminator :: d_loss: 0.645387], [ Generator :: loss: 0.621117]
epoch: 19908, [Discriminator :: d_loss: 0.638340], [ Generator :: loss: 0.846328]
epoch: 19909, [Discriminator :: d_loss: 0.619640], [ Generator :: loss: 0.792605]
epoch: 19910, [Discriminator :: d_loss: 0.576275], [ Generator :: loss: 0.860164]
epoch: 19911, [Discriminator :: d_loss: 0.564546], [ Generator :: loss: 1.004901]
epoch: 19912, [Discriminator :: d_loss: 0.647494], [ Generator :: loss: 0.949206]
epoch: 19913, [D